自然言語の扱い
---------------
- 自然言語を扱う以上単語を数値に変換し、なんらかのベクトル表現する必要がある
    - これを「自然言語の分散表現、または素性」という

### Bag of Words(BOW)
- 自然言語の分散表現のうち「文章」の分散表現でもっとも簡単なものに「Bag of Words」という表現がある
- BOWでは「どの単語が何回出現したか」という数で表す

### TF-IDF
- 単純に回数だけで表現するには荒すぎるため、単語に重要度を重み付けする手法
- 「同一文章に多く出てくる単語ほどその文章を特徴付ける」「その単語が様々な他の文章にも出現する場合、重要度を下げる」というルールを用いる
    - TF(Term Frequency)は単語の出現頻度を表す
    - IDF(Inverse Document Frequency)は単語が出現する文章頻度の逆数の対数を表す
- 次元数が多くなってしまう

### LSA(Latent Semantic Analysis)
- 文章 × 単語数のBOWの行列を特異値分解したのち低次元圧縮することで文章や単語を任意のn次元に圧縮する
- 特定のタスクではBOWやTF-IDF以上の性能を発揮した
- ベクトルのそれぞれの値が何を指しているか不明のため上手く行かなかった場合の原因が不明

### PLSA(Probabilistic Latent Semantic Analysis), LDA(LatentDirichletAllocation)
- トピックモデルと言われる
- トピックモデルでは文章や単語はその背後にトピックという隠れ変数をもっており、そのトピックの分布により確率的に生起されるという考え方
    - トピックとは政治、スポーツのようなジャンルのようなもの

### ニューラル言語モデル
- Word2Vecが有名
- ニューラルネットワークへの入力はデータセットの語彙数と同じ次元のワンホットベクトルになる
    - ※ ワンホットベクトルとは、ベクトルの内1箇所が1で残りが全て0のベクトル
- 単語の特徴をうまく捉えて分散表現ができる
- 訓練が非常に速い

Word2Vec
-------------------
- 青空文庫に収録されている小説の本文データを使用する
- 分散表現をTensorBordのEmbeddingsで可視化してみる

### データの前処理

------------------

### MeCab
- 日本語の形態素解析ライブラリ
- 形態素解析とは、文章を単語に分割して、各単語の品詞や活用形の解析を行うこと
- 英語などのアルファベットベースの言語ではスペースが区切りのため必要ないが日本語では必要

### MacでMecabをインストール

- brewでmecabをインストール

```
$ brew install mecab
$ brew install mecab-ipadic
```

- pythonのライブラリをインストール

```
$ pip install mecab-python3
または
$ pip install natto-py
```

- natto-pyは、FFI(Foreign function interface)というあるプログラム言語から他のプログラムを呼び出すための仕組み

#### 新語の辞書
https://github.com/neologd/mecab-ipadic-neologd/blob/master/README.ja.md


In [8]:
import glob
import re
import collections
import random

import numpy as np
from natto import MeCab

In [9]:
# ファイルの読み込み
def _read_docment(self, file_path):
    with open(file_path, 'r', encoding='sjis') as f:
        return f.read()

In [11]:
#ヘッダなどの不要データを前処理。必要な部分だけを返す。
def _preprocessing(self, document):

    lines = document.splitlines()
    processed_line = []

    horizontal_count = 0

    for line in lines:
        #ヘッダーは読み飛ばす
        if horizontal_count < 2:
            if line.startswith('-------'):
                horizontal_count += 1
            continue
        #フッターに入る行になったらそれ以降は無視
        if line.startswith('底本：'):
            break

        line = re.sub(r'《.*》', '', line) #ルビを除去
        line = re.sub(r'［.*］', '', line) #脚注を除去
        line =re.sub(r'[!-~]', '', line) #半角記号を除去
        line =re.sub(r'[︰-＠]', '', line) #全角記号を除去
        line = re.sub('｜', '', line) # 脚注の始まりを除去

        processed_line.append(line)

    return ''.join(processed_line)

In [13]:
#形態素解析
def _morphological(self, document):

    word_list = []
    # MeCabの形態素解析結果のフォーマット
    # %mが形態素のそのままの形 %f[o]が品詞、%f[1]が品詞の補足情報
    # %f[6]が基本形
    with MeCab('-F%f[0],%f[1],%f[6]') as mcb:
        for token in mcb.parse(document, as_nodes=True):
            features = token.feature.split(',')
            #名詞（一般）動詞（自立）、形容詞（自立）以外は除外
            if features[0] == '名詞' and features[1] == '一般' and features[2] != '':
                word_list.append(features[2])
            if features[0] == '動詞' and features[1] == '自立' and features[2] != '':
                word_list.append(features[2])
            if features[0] == '形容詞' and features[1] == '自立' and features[2] != '':
                word_list.append(features[2])
    return word_list

In [19]:
#辞書作成
def _build_data_sets(self, words, max_vocab):

    #単語出現回数を解析。出現数が少ないたんをUnknown wordとしてひとくくりに扱う
    word_frequency = [['UNW', -1]]
    # collections.Counterで単語の個数を計算
    word_frequency.extend(collections.Counter(words).most_common(max_vocab - 1))
    #単語=>IDの辞書
    w_to_id = dict()
    for word, _ in word_frequency:
        w_to_id[word] = len(w_to_id)
    #形態素解析した文章を単語IDの並びに変換
    id_sequence = list()
    unw_count = 0
    for word in words:
        #UNK処理
        if word in w_to_id:
            index = w_to_id[word]
        else:
            index = 0
            unw_count += 1
        id_sequence.append(index)
    word_frequency[0][1] = unw_count
    #単語ID=>単語の辞書
    id_to_w = dict(zip(w_to_id.values(), w_to_id.keys()))
    return id_sequence, word_frequency, w_to_id, id_to_w

### pythonメモ

#### extend(リストに別のリストを追加)

`<list>.extend(<list)`

#### counter#most_common

- カウンタの中で最も多い順にリストをソート結果のリストを返す

```
>>> Counter('abracadabra').most_common(3)  
[('a', 5), ('r', 2), ('b', 2)]
```